In [1]:
import os, sys
parent_folder_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_folder_path)

In [2]:
import pandas as pd

data_df = pd.read_pickle('../combined_countries/data/combined_countries_processed.pkl')
data_df.head()

,bookmark_count,bookmarked,conversation_id_str,created_at,display_text_range,entities,favorite_count,favorited,full_text,is_quote_status,...,MW_Reposts,MW_Comments,MW_Reactions,MW_Views,MW_Estimated Views,MW_Document Tags,post_type,url,has_retweeted_status_result,original_rest_id
3,0,False,1816217080397455588,Wed Jul 24 21:01:10 +0000 2024,"[0, 276]","{'hashtags': [], 'symbols': [], 'timestamps': ...",3,False,Algunos padres están alarmados por nuevas circ...,False,...,NaN,NaN,3.0,15.0,NaN,NaN,Post,https://twitter.com/1302389542457159685/status...,False,NaN
5,0,False,1816201806252490955,Wed Jul 24 20:39:44 +0000 2024,"[5, 72]","{'hashtags': [], 'symbols': [], 'timestamps': ...",0,False,@T13 .\nTodos extranjeros no pago vacuna algun...,False,...,NaN,NaN,NaN,1.0,NaN,NaN,Comment,https://twitter.com/15626578/status/1816211686...,False,NaN
6,0,False,1815161737496134018,Wed Jul 24 20:11:18 +0000 2024,"[53, 171]","{'hashtags': [], 'symbols': [], 'timestamps': ...",1,False,@Luis314PII @linea_progre @tasty4ever1 @esCesa...,False,...,NaN,NaN,NaN,9.0,NaN,NaN,Comment,https://twitter.com/78798396/status/1816204532...,False,NaN
7,0,False,1815161737496134018,Wed Jul 24 20:10:29 +0000 2024,"[53, 305]","{'hashtags': [], 'symbols': [], 'timestamps': ...",1,False,@Luis314PII @linea_progre @tasty4ever1 @esCesa...,False,...,NaN,NaN,1.0,4.0,NaN,NaN,Comment,https://twitter.com/78798396/status/1816204327...,False,NaN
11,0,False,1816165237101662691,Wed Jul 24 17:35:09 +0000 2024,"[0, 280]","{'hashtags': [], 'symbols': [], 'timestamps': ...",13,False,Estimados compañeros Bicentenarios.\nEste es u...,False,...,7.0,NaN,13.0,102.0,NaN,NaN,Post,https://twitter.com/106178430/status/181616523...,False,NaN


In [3]:
data_df.columns

Index(['bookmark_count', 'bookmarked', 'conversation_id_str', 'created_at',
       'display_text_range', 'entities', 'favorite_count', 'favorited',
       'full_text', 'is_quote_status', 'lang', 'quote_count', 'reply_count',
       'retweet_count', 'retweeted', 'retweeted_status_result', 'user_id_str',
       'status_id', 'extended_entities', 'possibly_sensitive',
       'possibly_sensitive_editable', 'in_reply_to_screen_name',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'note_tweet',
       'quoted_status_id_str', 'place', 'conversation_control', 'MW_Date',
       'MW_Time', 'MW_Document ID', 'MW_URL', 'MW_Input Name', 'MW_Keywords',
       'MW_Information Type', 'MW_Source Type', 'MW_Source Name',
       'MW_Source Domain', 'MW_Content Type', 'MW_Author Name',
       'MW_Author Handle', 'MW_Title', 'MW_Opening Text', 'MW_Hit Sentence',
       'MW_Image', 'MW_Hashtags', 'MW_Links', 'MW_Country', 'MW_Region',
       'MW_State', 'MW_City', 'MW_Language', 'MW_Sentiment

In [4]:
data_df.has_retweeted_status_result[:5]

3     False
5     False
6     False
7     False
11    False
Name: has_retweeted_status_result, dtype: bool

In [5]:
data_df.groupby(['post_type', 'MW_Country']).size()

post_type  MW_Country        
Comment    Argentina                 1
           Canada                    1
           Chile                     1
           Costa Rica              885
           Dominican Republic     2476
           Guatemala              1100
           Panama                 2797
           United States             2
Post       Costa Rica             4567
           Dominican Republic     9745
           Finland                   2
           Guatemala             10764
           India                     1
           Mexico                    2
           Panama                13887
           Uruguay                   1
dtype: int64

In [6]:
### Data Cleaning
print('Total rows', data_df.shape[0])

countries_list = ['Costa Rica', 'Dominican republic', 'Guatemala', 'Panama']
data_df = data_df[data_df.MW_Country.isin(countries_list)]
print('Total rows after filtering countries', data_df.shape[0])

data_df = data_df[data_df.full_text.notnull()]
print('Total rows after filtering null text', data_df.shape[0])

data_df = data_df.drop_duplicates(subset=['full_text'])
print('Total rows after removing full text duplicates', data_df.shape[0])

# save cleaned data
data_df.to_pickle('../combined_countries/data/cleaned_combined_countries_processed.pkl')

Total rows 46232
Total rows after filtering countries 34000
Total rows after filtering null text 34000
Total rows after removing full text duplicates 14761


In [7]:
import os
from dotenv import load_dotenv
from openai import OpenAI as OpenAIclient
from bertopic.backend import OpenAIBackend
from utils.embedding import precompute_embeddings

load_dotenv()

client = OpenAIclient(api_key=os.getenv('OPENAI_API_KEY'))
embedding_model = OpenAIBackend(client, 'text-embedding-3-small', batch_size=1000)
embedding_function = embedding_model.embed # has to be the function not the callable

# precompute embeddings
embeddings = precompute_embeddings(data_df = data_df,
                                   text_col_name='full_text',
                                   embedding_function=embedding_function,
                                   output_file_path='../data/combined_countries/cleaned_combined_countries.npy')

In [8]:
from utils.embedding import precompute_embeddings, read_embeddings

embeddings = read_embeddings('../data/combined_countries/cleaned_combined_countries.npy')
embeddings[:2]

array([[ 0.04628968,  0.01328233,  0.00646305, ...,  0.03969432,
        -0.00095864,  0.01252916],
       [ 0.02764316, -0.01576809,  0.01681836, ...,  0.02716704,
         0.0169724 ,  0.00551743]])

In [9]:
from bertopic import BERTopic
from nltk.corpus import stopwords
from openai import OpenAI as OpenAIclient
from bertopic.backend import OpenAIBackend
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired, OpenAI, MaximalMarginalRelevance

docs = data_df.full_text.to_list()

# hyperparameters from coherence and diversity based experimentation
params_list = [{'nr_topics': 15, 'min_topic_size': 15},
               {'nr_topics': 15, 'min_topic_size': 50},
               {'nr_topics': 15, 'min_topic_size': 100},
               {'nr_topics': 50, 'min_topic_size': 10},
               {'nr_topics': 100, 'min_topic_size': 100}]

# lists to save the two resulting models
models = []
topics = []
probs = []

# initialize custom model components
client = OpenAIclient(api_key=os.getenv('OPENAI_API_KEY'))
embedding_model = OpenAIBackend(client, 'text-embedding-3-small', batch_size=1000)
stop_words = stopwords.words(['english', 'spanish']) + ['https', 'http', 'www', 'com', 'org', 'net', 'edu', 'gov']
vectorizer_model = CountVectorizer(ngram_range=(1, 2),stop_words=stop_words)
main_representation = OpenAI(client, model='gpt-4o-2024-05-13', delay_in_seconds=5, chat=True)
keyword_representation = KeyBERTInspired()
keyword_sent_representation = [KeyBERTInspired(top_n_words=30), OpenAI(client, model='gpt-4o-2024-05-13', delay_in_seconds=10, chat=True)]


representation_model = {
   "Main": main_representation,
   "Keywords":  keyword_representation,
   "MMR Keywords":  keyword_sent_representation,
}

for params in params_list:

    model = BERTopic(
        embedding_model = embedding_model,
        vectorizer_model = vectorizer_model,
        representation_model = representation_model,
        nr_topics = params['nr_topics'],
        min_topic_size = params['min_topic_size'],
        calculate_probabilities = True,
        verbose = True
    )

    topic, prob = model.fit_transform(documents=docs, embeddings=embeddings)

    print(f'nr_topics:       {params["nr_topics"]}')
    print(f'min_topic_size:  {params["min_topic_size"]}')
    print(f'num_outliers:    {model.get_topic_info(-1)["Count"][0]}')

    # save results
    models.append(model)
    topics.append(topic)
    probs.append(prob)

2024-08-07 18:17:01,159 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2024-08-07 18:17:14,477 - BERTopic - Dimensionality - Completed ✓
2024-08-07 18:17:14,478 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-08-07 18:17:22,608 - BERTopic - Cluster - Completed ✓
2024-08-07 18:17:22,608 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 136/136 [24:32<00:00, 10.83s/it]
2024-08-07 18:56:01,592 - BERTopic - Representation - Completed ✓
2024-08-07 18:56:01,598 - BERTopic - Topic reduction - Reducing number of topics
100%|██████████| 15/15 [02:43<00:00, 10.88s/it]
2024-08-07 19:00:19,398 - BERTopic - Topic reduction - Reduced number of topics from 136 to 15
2024-08-07 19:00:19,518 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


nr_topics:       15
min_topic_size:  15
num_outliers:    6090


2024-08-07 19:00:25,099 - BERTopic - Dimensionality - Completed ✓
2024-08-07 19:00:25,101 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-08-07 19:00:25,906 - BERTopic - Cluster - Completed ✓
2024-08-07 19:00:25,907 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 8/8 [01:26<00:00, 10.84s/it]
2024-08-07 19:02:47,688 - BERTopic - Representation - Completed ✓
2024-08-07 19:02:47,701 - BERTopic - Topic reduction - Reducing number of topics
2024-08-07 19:02:47,703 - BERTopic - Topic reduction - Reduced number of topics from 8 to 8
2024-08-07 19:02:47,785 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


nr_topics:       15
min_topic_size:  50
num_outliers:    100


2024-08-07 19:02:52,752 - BERTopic - Dimensionality - Completed ✓
2024-08-07 19:02:52,754 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-08-07 19:02:53,770 - BERTopic - Cluster - Completed ✓
2024-08-07 19:02:53,770 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 8/8 [01:25<00:00, 10.64s/it]
2024-08-07 19:05:10,647 - BERTopic - Representation - Completed ✓
2024-08-07 19:05:10,651 - BERTopic - Topic reduction - Reducing number of topics
2024-08-07 19:05:10,653 - BERTopic - Topic reduction - Reduced number of topics from 8 to 8
2024-08-07 19:05:10,757 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


nr_topics:       15
min_topic_size:  100
num_outliers:    515


2024-08-07 19:05:15,702 - BERTopic - Dimensionality - Completed ✓
2024-08-07 19:05:15,704 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-08-07 19:05:35,345 - BERTopic - Cluster - Completed ✓
2024-08-07 19:05:35,345 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 217/217 [40:14<00:00, 11.13s/it]
2024-08-07 20:08:22,199 - BERTopic - Representation - Completed ✓
2024-08-07 20:08:22,213 - BERTopic - Topic reduction - Reducing number of topics
100%|██████████| 50/50 [08:57<00:00, 10.75s/it]
2024-08-07 20:22:31,502 - BERTopic - Topic reduction - Reduced number of topics from 217 to 50
2024-08-07 20:22:31,750 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


nr_topics:       50
min_topic_size:  10
num_outliers:    6464


2024-08-07 20:22:38,067 - BERTopic - Dimensionality - Completed ✓
2024-08-07 20:22:38,068 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-08-07 20:22:39,053 - BERTopic - Cluster - Completed ✓
2024-08-07 20:22:39,053 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 8/8 [01:27<00:00, 10.93s/it]
2024-08-07 20:25:16,029 - BERTopic - Representation - Completed ✓
2024-08-07 20:25:16,034 - BERTopic - Topic reduction - Reducing number of topics
2024-08-07 20:25:16,035 - BERTopic - Topic reduction - Reduced number of topics from 8 to 8


nr_topics:       100
min_topic_size:  100
num_outliers:    683


In [11]:
pd.set_option('max_colwidth', 800)

for idx,model in enumerate(models):
    print(model.get_topic_info()[['Topic', 'Count', 'Name', 'Keywords', 'MMR Keywords']])
    model.get_topic_info()[['Topic', 'Count', 'Name', 'Keywords']].to_csv(f'/Users/wesamazaizeh/Documents/pfizer_slt/combined_countries/results/topics_model_{idx}.csv', index=False)

    Topic  Count  \
0      -1   6090   
1       0   6873   
2       1    591   
3       2    342   
4       3    334   
5       4    103   
6       5     94   
7       6     84   
8       7     70   
9       8     61   
10      9     37   
11     10     26   
12     11     23   
13     12     17   
14     13     16   

                                                                                                                             Name  \
0                                                                                    -1_Vaccine Criticism and Pandemic Skepticism   
1                                                                           0_Vacunación contra el COVID-19 y efectos secundarios   
2                                                                              1_Ley de Extinción de Dominio y Corrupción Pública   
3                                                                                           2_HPV Vaccine Awareness and Promotion   
4             

In [16]:
models[4].visualize_topics()

In [18]:
for idx, model in enumerate(models):
    try:
        hierarchical_topics = model.hierarchical_topics(docs)
        tree = model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)
        tree.show()
        try:
            tree.write_html(f'/Users/wesamazaizeh/Documents/pfizer_slt/combined_countries/results/hierarchical_tree_{idx}.html')
        except FileNotFoundError:
            os.mkdir('/Users/wesamazaizeh/Documents/pfizer_slt/combined_countries/results')
            tree.write_html(f'/Users/wesamazaizeh/Documents/pfizer_slt/combined_countries/results/hierarchical_tree_{idx}.html')
    except Exception as e:
        print(f'Failed to visualize hierarchical tree for model {idx}\n {e}')

  0%|          | 0/13 [00:00<?, ?it/s]


Failed to visualize hierarchical tree for model 0
 dimension mismatch


  0%|          | 0/6 [00:00<?, ?it/s]


Failed to visualize hierarchical tree for model 1
 dimension mismatch


  0%|          | 0/6 [00:00<?, ?it/s]


Failed to visualize hierarchical tree for model 2
 dimension mismatch


  0%|          | 0/48 [00:00<?, ?it/s]


Failed to visualize hierarchical tree for model 3
 dimension mismatch


100%|██████████| 6/6 [00:34<00:00,  5.83s/it]
